Section 1 - Import Dependencies

In [1]:
#Import Libraries & Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
import gmaps
import scipy.stats as sts
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from config import api_key_ggle

%matplotlib inline

Import files

In [2]:
#Load CSV containing from weatherpy out put
path = os.path.join('..','WeatherPy','Weatherhome.csv')
data_df = pd.read_csv(path)
data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,41.00,87,20,6.91,AR,1613266121
1,Bluff,-46.6000,168.3333,56.59,84,100,5.50,NZ,1613266116
2,Rikitea,-23.1203,-134.9692,80.17,73,97,10.45,PF,1613266201
3,Punta Arenas,-53.1500,-70.9167,44.60,70,40,5.75,CL,1613265907
4,Vaini,-21.2000,-175.2000,86.00,74,20,10.36,TO,1613266201


In [3]:
# configure gmaps with API Key
gmaps.configure(api_key=api_key_ggle)

In [4]:
#store 'Lat' and 'Lng' into locations
locations = data_df[['Lat', 'Lng']].astype(float)
#drop any NaN values in Humidity column
#hum = data_df.dropna()
humidity = data_df["Humidity"].astype(float)
#humidity = hum['Humidity'].astype(float)
#data_df = data_df.dropna()
humidity

0       87.0
1       84.0
2       73.0
3       70.0
4       74.0
       ...  
546     69.0
547    100.0
548     85.0
549     92.0
550     90.0
Name: Humidity, Length: 551, dtype: float64

In [6]:
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 3,)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
locations

,Lat,Lng
0,-54.8000,-68.3000
1,-46.6000,168.3333
2,-23.1203,-134.9692
3,-53.1500,-70.9167
4,-21.2000,-175.2000
...,...,...
546,18.4167,-64.6167
547,36.4615,-77.6542
548,-24.5561,-54.0567
549,68.4140,15.9951


In [16]:
#Configure Gmaps and narrow down the DataFrame to find ideal weather conditions
#Drop any rows that don't contain all three conditions

condition = data_df.loc[(data_df['Max Temp']<80) & (data_df['Max Temp']>70)\
                & (data_df['Wind Speed']<10)\
                        &(data_df['Cloudiness']==0)]
condition

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,Saint-Leu,-21.1500,55.2833,73.99,60,0,5.23,RE,1613266204
35,Busselton,-33.6500,115.3333,73.00,61,0,3.00,AU,1613266209
79,Port Elizabeth,-33.9180,25.5701,73.40,94,0,6.91,ZA,1613266217
83,Bubaque,11.2833,-15.8333,77.92,63,0,8.90,GW,1613266217
90,Banikoara,11.2985,2.4386,72.90,18,0,4.63,BJ,1613266218
116,Ayorou,14.7308,0.9174,76.01,16,0,8.55,NE,1613266222
123,New Norfolk,-42.7826,147.0587,71.01,52,0,4.00,AU,1613266114
225,Calderitas,18.5500,-88.2500,77.25,81,0,6.20,MX,1613266236
229,Sémbé,1.6481,14.5806,74.41,77,0,3.31,CG,1613266237
283,Umluj,25.0213,37.2685,74.77,56,0,6.17,SA,1613266243


In [17]:
condition.shape

(18, 9)

In [31]:
cloudc

,Lat,Lng
17,24.6877,46.7219
18,-21.1500,55.2833
25,-33.8667,121.9000
35,-33.6500,115.3333
56,65.8500,74.4859
...,...,...
504,20.9310,-17.0347
507,65.8167,87.9833
509,61.9500,159.2333
515,34.9072,48.4414


In [ ]:
#Define Figure Layout
figure_layout = {
    
    'width':'1000px',
    'height':'600px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
    }

#Define Symbol Layers
f = gmaps.figure(layout=figure_layout, center=(33.7490, -84.386330),zoom_level=13, map_type='TERRAIN')
f.add_layer(gmaps.symbol_layer(coordg1, fill_color=(244, 27, 63, 0.8), stroke_color=(244, 27, 63, 0.8), scale=2))
f.add_layer(gmaps.symbol_layer(coordg2, fill_color=(38, 156, 98, 0.8), stroke_color=(38, 156, 98, 0.8), scale=2)

In [ ]:
#Explore the Dataset
df.info()

In [ ]:

#We will only focus on crime happenning in year 2016
crime = df.loc[df['Year']=='2016']
crime.reset_index(drop =True, inplace=True)
crime.head()

In [ ]:

#We will pick as our groups 5 unrelated types of crime
#1. LARCENY-FROM VEHICLE
#2. BURGLARY-RESIDENCE
#3. ROBBERY-PEDESTRIAN
#4. RAPE
#5. HOMICIDE
crime['crime'].value_counts()

In [ ]:

#Group 1: LARCENY-FROM VEHICLE
g1 = crime.loc[crime['crime']=='LARCENY-FROM VEHICLE']
g1.reset_index(drop =True, inplace=True)
g1.drop(['number', 'date', 'location', 'beat', 'neighborhood', 'npu',
       'lat', 'long','Year'], axis=1, inplace=True)
g1w = g1.groupby(['Month','Day']).count()
g1w.rename(columns={'crime':'Crime Count'},inplace=True)
g1w.reset_index(inplace=True)

In [ ]:
#Parameter for the API call, units will be handled in Metric system
params = {'key':vcwkey, 'location':location, 'aggregateHours':aggregateHours,
          'unitGroup':unitGroup,'contentType':contentType}
#**************************************************************************************************************
#Group 1: LARCENY-FROM VEHICLE

for i, row in g1w.iterrows():
    params['startDateTime'] = '2016-'+row['Month']+'-'+row['Day']+'T00:00:00'
    params['endDateTime'] = '2016-'+row['Month']+'-'+row['Day']+'T23:00:00'
    response = requests.get(base_url,params=params).json()
    
    try:
        g1w.loc[i,'Avg_Temp']= response['locations']['Atlanta']['values'][0]['temp']
        g1w.loc[i,'Humidity']= response['locations']['Atlanta']['values'][0]['humidity']
        g1w.loc[i,'Precip']= response['locations']['Atlanta']['values'][0]['precip']
        g1w.loc[i,'Wind_Speed']= response['locations']['Atlanta']['values'][0]['wspd']
        g1w.loc[i,'Conditions']= response['locations']['Atlanta']['values'][0]['conditions']
    except:
        g1w.loc[i,['Avg_Temp','Humidity','Precip','Wind_Speed','Conditions']] = np.nan

In [ ]:
#Concatenate and Convert the latitutude/longitude columns into a single column for aggregation
cdf2['latLong']= cdf2['lat'].astype(str) + "," + cdf2['long'].astype(str)

In [ ]:

#Create data frame to hold the unique long/lat values
cityLL =pd.DataFrame(cdf2['latLong'].unique())

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
radius = "1609.34" # 1 mile proximity converted to meters
max_price = "2" # maximum price set to level 2

    #location = 

params = {
"radius": radius,
"query": ["ice cream shop"],
"type": ['cafe','restaurant','drugstore'],
"maxprice": max_price,
"key": gkey,
}

for i, row in cityLL.iterrows():
    
    params["location"] = row[0]
    response = requests.get(base_url, params=params).json()
    try:
        cityLL.loc[i,'store count']= len(response['results'])
        #print(len(response['results']))
        #print(row[0])
    except:
        cityLL.loc[i,'store count']= np.nan

In [ ]:

#Data frame created to tie the coordinates to the store count based on 1 mile proximity
cityLL

In [ ]:

cityLL.rename(columns={0: "latLong"},inplace=True)

In [ ]:

#merging the total crime dataframe on latLong column
cdf3 = cdf2.merge(cityLL, on='latLong')

In [ ]:

#Convert Store count to integer
cdf3['store count']=cdf3['store count'].astype(int)

In [ ]:
#Group the Atlanta all crimes data frame by store count
cdf3.groupby('date').sum()